In [1]:
import os
import sys

TOP_DIR=os.path.realpath("../../..")
if TOP_DIR not in sys.path: sys.path.append(TOP_DIR)

In [2]:
import pandas as pd
import numpy as np
from lib.util.convert import standardise_columns, literal_converter
from lib.util.geo import normalise_leeds_wards, ward_name_to_code

In [3]:
data = pd.read_csv('../../../working/metrics/community/events.csv', parse_dates=['Start date'])
data = data.rename(columns=standardise_columns)

In [4]:
wards = data['ward_(from_venue)'].fillna('[]').pipe(literal_converter).apply(np.unique)

In [5]:
data['ward_name'] = wards.apply(lambda x: x[0] if len(x) > 0 else np.nan).pipe(normalise_leeds_wards)
data['ward_code'] = data.ward_name.pipe(ward_name_to_code)
data['audience'] = data.actual_audience_size_number_of_participants_in_person
cleaned_data = data.loc[:, ['event_name', 'project_name', 'start_date', 'ward_name', 'ward_code', 'audience']]

In [6]:
DATA_DIR='../../../data/metrics/community'
os.makedirs(DATA_DIR, exist_ok=True)
cleaned_data.to_csv(os.path.join(DATA_DIR, 'events.csv'), index=False)

Missing or duplicated wards

In [7]:
data.loc[wards.apply(len) != 1, ['project_name', 'event_name', 'start_date', 'ward_(from_venue)']]

,project_name,event_name,start_date,ward_(from_venue)
30,01 - The Awakening,Creation Station - Leeds City Council,2022-11-23,NaN
31,01 - The Awakening,Creation Station - Leeds City Council,2022-11-24,NaN
49,01 - The Awakening,Leeds Arts University - Creation Station,NaT,NaN
53,01 - The Awakening,Light Night - Creation Station,2023-10-13,NaN
54,01 - The Awakening,Lotherton Hall,2023-10-23,NaN
...,...,...,...,...
262,Hidden Stories,Edible Archives: Eating The Corn Exchange Invi...,2023-10-29,NaN
263,Hidden Stories,Edible Archives: Eating The Corn Exchange Slot 1,2023-10-29,NaN
264,Hidden Stories,Edible Archives: Eating The Corn Exchange Slot 2,2023-10-29,NaN
265,Hidden Stories,Harewood House - Aberford Community Cafe,2023-04-28,NaN


Missing audience

In [8]:
data.loc[data.audience.isna(), ['project_name', 'event_name', 'start_date', 'audience']]

,project_name,event_name,start_date,audience
57,01 - The Awakening,Tetley - Creation Station,NaT,NaN
132,12 - My LEEDS 2023,Adel and Wharfdale Grant Info Session 1,2023-02-20,NaN
133,12 - My LEEDS 2023,Adel and Wharfdale Grant Info Session 2,2023-02-22,NaN
135,12 - My LEEDS 2023,Alwoodley Grant Info Session 1,2023-02-21,NaN
136,12 - My LEEDS 2023,Alwoodley Grant Info Session 2,2023-02-21,NaN
146,12 - My LEEDS 2023,Ardsley and Robin Hood Grant Info Session 1,2023-02-13,NaN
147,12 - My LEEDS 2023,Armley Grant Info Session 2,2023-02-15,NaN
148,12 - My LEEDS 2023,Armley Grant Info Session 3,2023-02-17,NaN
150,12 - My LEEDS 2023,Bramley and Stanningley Grant Info Session 1,2023-02-09,NaN
151,12 - My LEEDS 2023,Bramley and Stanningley Grant Info Session 2,2023-02-21,NaN
